In [27]:
import json


with open('./json/file_new_stopwords_token.json') as data_file:
    dataNormal = json.load(data_file)

In [28]:
# load right dataset
dataIteration = dataNormal

def filterNonEnglishOut(dataIteration):
    dataIteration_preprocess = []

    words_order = dataIteration
    only_english_words = []

    # to keep only english words
    with open("./wordlist_cz.txt") as word_file:
        only_english_words = set(word.strip().lower() for word in word_file)

    for line in words_order:
        english_line = ""
        for w in line.split():
            if w.strip().lower() in english_words_file:
                english_line += str(w.strip().lower()) + " "
            else: 
                pass
        dataIteration_preprocess.append(english_line)    
    return dataIteration_preprocess

In [97]:
reviewWords = ""

words = []

labels = []

treashold = 1

passCounter = 0

#every review
for x in range(0,len(dataIteration)):
    
    reviewWords = ""
    
    for review in  dataIteration[x].get("review"):
        reviewWords += review + " "
        

    if(dataIteration[x].get("usefulReview") > dataIteration[x].get("uselessReview") + treashold):
        labels.append(1)
        words.append(reviewWords)
#         labels.append(1)
#         words.append(reviewWords)
#         labels.append(1)
#         words.append(reviewWords)
#         labels.append(1)
#         words.append(reviewWords)
#         labels.append(1)
#         words.append(reviewWords)
#         labels.append(1)
#         words.append(reviewWords)
#         labels.append(1)
#         words.append(reviewWords)
    elif (dataIteration[x].get("usefulReview") - treashold < dataIteration[x].get("uselessReview")):
        if(x % 5 == 0):
            labels.append(-1)
            words.append(reviewWords)
    else :
        passCounter += 1

In [99]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(words, labels, test_size=0.2, random_state=0)

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

words_train_count = count_vect.fit_transform(X_train)

In [100]:
len (count_vect.vocabulary_.items())

10753

In [13]:
freqs = [(word, words_train_count.getcol(idx).sum()) for word, idx in count_vect.vocabulary_.items()]
#sort from largest to smallest
print (sorted (freqs, key = lambda x: -x[1])[:10])

[('doporučovat', 3135), ('dobrý', 2518), ('cena', 1898), ('spokojený', 1828), ('obchod', 1443), ('zboží', 1288), ('velmi', 1127), ('dno', 1121), ('dna', 1069), ('rychlý', 1023)]


In [101]:
from sklearn.feature_extraction.text import TfidfTransformer

tf_transformer = TfidfTransformer(use_idf=False).fit(words_train_count)
words_train_tf = tf_transformer.transform(words_train_count)

tfidf_transformer = TfidfTransformer()
words_train_tfidf = tfidf_transformer.fit_transform(words_train_tf)

In [90]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(words_train_tfidf, y_train)

from sklearn.feature_selection import SelectFromModel
# Set a minimum threshold of 0.25

sfm = SelectFromModel(clf , prefit=True)
print (words_train_tfidf.shape)
words_train_tfidf = sfm.transform(words_train_tfidf)
print (words_train_tfidf.shape)

(16236, 18471)
(16236, 13248)


In [91]:
n_features

19221

In [103]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(words_train_tfidf, y_train)
docs_new = X_test
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

print("Score " + str(clf.score(X_new_tfidf, y_test)))

from sklearn.metrics import f1_score
predicted = clf.predict(X_new_tfidf)
# F1 = 2 * (precision * recall) / (precision + recall) 
print (f1_score(y_pred=predicted, y_true=y_test, average="macro"))
print (f1_score(y_pred=predicted, y_true=y_test, average="micro"))
print (f1_score(y_pred=predicted, y_true=y_test, average="weighted"))


Score 0.734939759036
0.567795518207
0.734939759036
0.674160035098


In [104]:
from sklearn.metrics import roc_auc_score

predicted_proba = clf.predict_proba(X_new_tfidf)
# Compute Area Under the Curve (AUC) from prediction scores
print ("ROC :" + str(roc_auc_score(y_test, predicted_proba[:, 1])))

ROC :0.727250507174


In [98]:
print (len([l for l in labels if l == 1])/len(labels))
print (len([l for l in labels if l == -1]))
print (len([l for l in labels if l == 1]))

0.3078064157240868
3733
1660


In [ ]:
from __future__ import print_function

from pprint import pprint
from time import time
import logging

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Sample pipeline for text feature extraction and evaluation
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer='word')),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

with open('./czech_stop_words.txt') as stop_words:
    stop_word_list = list([f.strip() for f in stop_words])

# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__stop_words' : (stop_word_list ,'english', None),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'vect__analyzer': ('word', 'char', 'char_wb') ,
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    #'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}

# multiprocessing requires the fork to happen in a __main__ protected
# block

# find the best parameters for both the feature extraction and the
# classifier
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(X_test, y_test)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))